In [19]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from fancyimpute import KNN
from sklearn.metrics import mean_squared_error
from math import sqrt
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima import auto_arima
import sys
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.stattools import adfuller
from openpyxl import load_workbook
import warnings
warnings.filterwarnings("ignore")

In [27]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    #print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    #print(out.to_string())         
    
    if result[1] <= 0.05:
        return "Stationary"
    else:
        return "Non-Stationary"

In [26]:
df = pd.read_excel(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", sheet_name='Version2',engine='openpyxl',index_col='Date',parse_dates=True)
df.head()

,Money Market - Net liquidity injected,Money Market - Call Money,Money Market - Triparty Money,Money Market - Market Repo,Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot,Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward,Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward Cancel,...,NSDL FPI - D-PMO,NSDL FPI - D-ST,NSDL FPI - DVRR_SE,NSDL FPI - DVRR-PMO,NSDL FPI - DVRR-ST,NSDL FPI - H_SE,NSDL FPI - H-PMO,NSDL FPI - H-ST,NSDL FPI - Total,Value of Rupee
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-14,-2.915450,4.64,4.27,4.92,2297.000000,861.000000,488.000000,47.000000,89,36.000000,...,-0.210000,453.720000,-0.21,41.77,41.56,0.650000,0.0,0.65,588.68,71.45
2020-02-15,-3.093000,4.81,4.85,4.92,1336.838723,586.685775,292.363524,-42.635685,44,-5.862873,...,8.755250,524.714736,0.00,0.00,0.00,1.223889,0.0,0.55,25.64,71.45
2020-02-16,-3.093130,4.93,4.85,4.92,671.660449,340.320116,113.026919,-65.615348,44,2.857913,...,6.797701,391.039830,0.00,0.00,0.00,1.008416,0.0,0.55,25.64,71.45
2020-02-17,-2.824517,4.93,4.85,4.92,1714.000000,509.000000,356.000000,60.000000,44,35.000000,...,2.240000,185.070000,0.00,0.00,0.00,0.550000,0.0,0.55,25.64,71.38
2020-02-18,-2.597337,4.95,4.89,4.88,3017.000000,739.000000,735.000000,210.000000,61,39.000000,...,-0.610000,-16.580000,0.00,5.60,5.60,0.330000,0.0,0.33,13.61,71.55


In [28]:
stationary = []
non_stationary = []
columns = {}
methods = {}
for i in list(df.columns):
    result = adf_test(df[i],i)
    if result == "Stationary":
        stationary.append(i)
        columns[i] = "Stationary"
    else:
        non_stationary.append(i)
        columns[i] = "Non - Stationary"

non_stationary.remove('Value of Rupee')

len(non_stationary)

28

In [29]:
from sklearn.linear_model import LinearRegression
lr = []
for i in non_stationary:
    dff = df[[i]]
    X = np.array(range(len(dff)))
    X = X.reshape(-1, 1)
    y = dff.values
    model = LinearRegression()
    model.fit(X, y)
    trend = model.predict(X)
    dff['nontrend'] = y - trend
    result = adf_test(dff['nontrend'])
    if result == "Stationary":
        lr.append(i)
        df[i] = dff['nontrend'].tolist()
        methods[i] = 'Detrending'


In [31]:
print(lr)
print(len(lr))

['Money Market - Triparty Money', 'Money Market - Market Repo', 'FX_Reserves -Gold', 'Outward Remittances', 'Foreign Trade - Trade Balance - Oil', 'Sales and Purchases of Dollar -Sale ', 'International Trade Services - Imports', 'International Trade Services - Exports', 'NSE Open Interest - Option Index Call Long', 'NSE Open Interest - Option Index Put Long', 'NSE Open Interest - Option Index Call Short', 'NSE Open Interest - Option Index Put Short']
12


In [32]:
stationary = []
non_stationary = []
for i in list(df.columns):
    result = adf_test(df[i],i)
    if result == "Stationary":
        stationary.append(i)
    else:
        non_stationary.append(i)

non_stationary.remove('Value of Rupee')

len(non_stationary)

16

In [33]:
from statsmodels.tsa.seasonal import STL
stl_ans = []
df3 = df.copy()
for i in non_stationary:
    df4 = df3[[i]]
    stl = STL(df4[df4.columns[0]], period=12)
    res = stl.fit()
    df4['trend'] = res.trend
    df4['seasonal'] = res.seasonal
    df4['residual'] = res.resid
    result = adf_test(df4['residual'])
    if result == "Stationary":
        stl_ans.append(i)
        df[i] = df4['residual'].tolist()
        methods[i] = 'STL Decomposition'

In [34]:
stl_ans

['Money Market - Net liquidity injected',
 'Money Market  - Call Money',
 'FX_Reserves - Total Reserves',
 'FX_Reserves -Foreign Currency',
 'NEER AND REER - Trade NEER',
 'NEER AND REER - Trade REER',
 'NEER AND REER - Export NEER',
 'NEER AND REER - Export REER',
 'Foreign Trade - Trade Balance',
 'Foreign Trade - Trade Balance - Non Oil',
 'Maturity Breakdown - Up to One Month',
 'Maturity Breakdown - One to Three Months',
 'Maturity Breakdown - Three Months to One Year',
 'Maturity Breakdown - More than a Year',
 'Maturity Breakdown - Total',
 'Sales and Purchases of Dollar - Purchase']

In [35]:
stationary = []
non_stationary = []
for i in list(df.columns):
    result = adf_test(df[i],i)
    if result == "Stationary":
        stationary.append(i)
    else:
        non_stationary.append(i)

non_stationary.remove('Value of Rupee')

len(non_stationary)

0

In [36]:
methods

{'Money Market - Triparty Money': 'Detrending',
 'Money Market - Market Repo': 'Detrending',
 'FX_Reserves -Gold': 'Detrending',
 'Outward Remittances': 'Detrending',
 'Foreign Trade - Trade Balance - Oil': 'Detrending',
 'Sales and Purchases of Dollar -Sale ': 'Detrending',
 'International Trade Services - Imports': 'Detrending',
 'International Trade Services - Exports': 'Detrending',
 'NSE Open Interest - Option Index Call Long': 'Detrending',
 'NSE Open Interest - Option Index Put Long': 'Detrending',
 'NSE Open Interest - Option Index Call Short': 'Detrending',
 'NSE Open Interest - Option Index Put Short': 'Detrending',
 'Money Market - Net liquidity injected': 'STL Decomposition',
 'Money Market  - Call Money': 'STL Decomposition',
 'FX_Reserves - Total Reserves': 'STL Decomposition',
 'FX_Reserves -Foreign Currency': 'STL Decomposition',
 'NEER AND REER - Trade NEER': 'STL Decomposition',
 'NEER AND REER - Trade REER': 'STL Decomposition',
 'NEER AND REER - Export NEER': 'STL D

In [37]:
import pandas as pd
from openpyxl import load_workbook
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Version3'
df.index = df.index.strftime('%d-%m-%Y')
df.to_excel(writer, sheet_name=sheet_name, index=True)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")

In [38]:
df3 = pd.DataFrame.from_dict(columns,orient='index').reset_index()
df3.columns = ['Column','Type']
df3.set_index('Column')
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Report - Version3'
df3.to_excel(writer, sheet_name=sheet_name, index=True,startrow=3, startcol=0)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
df3

,Column,Type
0,Money Market - Net liquidity injected,Non - Stationary
1,Money Market - Call Money,Non - Stationary
2,Money Market - Triparty Money,Non - Stationary
3,Money Market - Market Repo,Non - Stationary
4,Foreign Exchange Turnover (Purchases) - Mercha...,Stationary
...,...,...
66,NSDL FPI - H_SE,Stationary
67,NSDL FPI - H-PMO,Stationary
68,NSDL FPI - H-ST,Stationary
69,NSDL FPI - Total,Stationary


In [40]:
df4 = pd.DataFrame.from_dict(methods,orient='index').reset_index()
df4.columns = ['Variable','Method']
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Report - Version3'
df4.to_excel(writer, sheet_name=sheet_name, index=True,startrow=3, startcol=5)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
df4

,Variable,Method
0,Money Market - Triparty Money,Detrending
1,Money Market - Market Repo,Detrending
2,FX_Reserves -Gold,Detrending
3,Outward Remittances,Detrending
4,Foreign Trade - Trade Balance - Oil,Detrending
5,Sales and Purchases of Dollar -Sale,Detrending
6,International Trade Services - Imports,Detrending
7,International Trade Services - Exports,Detrending
8,NSE Open Interest - Option Index Call Long,Detrending
9,NSE Open Interest - Option Index Put Long,Detrending
